In [42]:
import tensorflow as tf

In [66]:
#!pip install ipadic

from transformers import BertJapaneseTokenizer, TFBertForSequenceClassification

model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
def preprocess_text(text):
    encoded_text = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding="max_length",
        return_tensors="tf"
    )
    input_ids = encoded_text["input_ids"]
    attention_mask = encoded_text["attention_mask"]
    return input_ids, attention_mask

In [62]:
text = '猫がかわいです'
tokenizer.tokenize(text)

['猫', 'が', 'かわい', 'です']

In [63]:
encoding = tokenizer.encode_plus(text)

input_ids = encoding['input_ids']
tokenizer.decode(input_ids)

'[CLS] 猫 が かわい です [SEP]'

In [60]:
encoding

{'input_ids': [2, 13221, 9, 20653, 14, 22167, 2992, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [53]:
input_ids, attention_mask = preprocess_text(sentence)
model(input_ids, attention_mask=attention_mask)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.17598309, 0.06549294]], dtype=float32)>, hidden_states=None, attentions=None)

In [45]:
model.summary()

Model: "tf_bert_for_sequence_classification_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_265 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 110618882 (421.98 MB)
Trainable params: 110618882 (421.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [77]:
model.predict(preprocess_text(text))

1/1 [==============================] - 1s 1s/step


TFSequenceClassifierOutput(loss=None, logits=array([[-0.14946349, -0.6031715 ]], dtype=float32), hidden_states=None, attentions=None)

In [78]:
from transformers import TFBertModel
bertModel = TFBertModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [79]:
bertModel.summary()

Model: "tf_bert_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109482240 (417.64 MB)
Trainable params: 109482240 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [86]:
output = bertModel.predict(preprocess_text(text))
output.last_hidden_state.shape, output.pooler_output.shape

1/1 [==============================] - 1s 1s/step


((1, 512, 768), (1, 768))